In [57]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm


In [2]:
# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [15]:
# df = pd.read_csv("./HYPO.tsv", sep='\t')

# # Extract input texts from the "HYPO" column
# texts = df['HYPO'].tolist()
# paraphrases = df['PARAPHRASES'].tolist()
# minimal_units = df['MINIMAL UNITS CORPUS'].tolist()

# features = ['HYPO', "PARAPHRASES", "MINIMAL UNITS CORPUS"]
# # print(texts)
# # Combine features
# combined_data = {'HYPO': texts, "PARAPHRASES": paraphrases, "MINIMAL UNITS CORPUS": minimal_units}
# print(combined_data)


# # Extract labels
# labels = [1] * len(texts)  # Assuming all examples in "HYPO" are exaggerations

# # Initialize BERT tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# # Tokenize and encode input data
# inputs = tokenizer(combined_texts, padding=True, truncation=True, return_tensors='pt')

# # Convert labels to tensor
# labels = torch.tensor(labels)

# # Create TensorDataset
# dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels)

# # Initialize BERT model
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# # Define optimizer
# optimizer = AdamW(model.parameters(), lr=2e-5)

# # Create DataLoader
# dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# # Train the model
# for epoch in range(3):  # Adjust number of epochs as needed
#     for batch in dataloader:
#         input_ids, attention_mask, labels = batch
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()

# Save or deploy the model for inference

{'HYPO': ["The principal is unhappy...we're cooked. ", 'The availability of data is likely to lead to a veritable orgy of statistical analysis. ', 'Love you to the moon and back. ', 'Man, your compassion is greater than space.', 'Marriage is the grave of love.', 'I went into the shop and we cleared the shelves out. ', "He's as mad as a hippo with a hernia. ", "You're making a mountain out of a molehill. ", 'Speech is silver, but silence is golden. ', "No offence but she's out of the world dumb.", "Miss Trunchbull's lips stretched in a glacial smile.", "We're going to turn this sow's ear into a silk purse.", "This one is valuable: you don't touch a hair of his head.", 'He has a watermelon instead of a belly. ', 'The argument with my bestfriend turned into a tempest in a teacup.', "You know, the world could end before I'd let something happen to you.", 'We must fight with the force of a typhoon.', 'These people would follow you to the end of the earth. ', 'Beauty draws with a single hair

In [37]:
# Load the data
data = pd.read_csv('HYPO.tsv', sep='\t', names=['HYPO', 'PARAPHRASES', 'MINIMAL UNITS CORPUS'])
data['label-1'] = [1] * len(data['HYPO'])  # Assuming all examples in 'HYPO' are exaggerations
data['label-0'] = [0] * len(data["MINIMAL UNITS CORPUS"]) # assuming all minimal corpus examples are not exaggerations

data['text'] = pd.concat([data['HYPO'], data['MINIMAL UNITS CORPUS']], ignore_index=True)
# Create labels (1 for hyperbole, 0 for non-hyperbole)
data['label'] = pd.concat([data['label-1'], data['label-0']], ignore_index=True)

print(data['text'])
print(data['label'])

0                                                   HYPO
1              The principal is unhappy...we're cooked. 
2      The availability of data is likely to lead to ...
3                        Love you to the moon and back. 
4            Man, your compassion is greater than space.
                             ...                        
705                          Your room is a jungle!     
706           I am so tired I could sleep for a year.   
707                      Every flavor is dynamite.      
708                She is sinking in a sea of misery.   
709                   We all cracked up at the joke.    
Name: text, Length: 710, dtype: object
0      1
1      1
2      1
3      1
4      1
      ..
705    1
706    1
707    1
708    1
709    1
Name: label, Length: 710, dtype: int64


In [39]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
inputs = tokenizer(data['text'].tolist(), padding=True, truncation=True, return_tensors='pt')
labels = torch.tensor(data['label'].tolist())

In [45]:
# Assuming inputs is a dictionary containing tokenized input data
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

# Split the input data and labels
train_input_ids, val_input_ids, train_attention_mask, val_attention_mask, train_labels, val_labels = train_test_split(
    input_ids, attention_mask, labels, test_size=0.2, random_state=42)

# Optionally, you can combine the split input data into a dictionary again
train_inputs = {'input_ids': train_input_ids, 'attention_mask': train_attention_mask}
val_inputs = {'input_ids': val_input_ids, 'attention_mask': val_attention_mask}

print(train_inputs)

{'input_ids': tensor([[  101,  2635,  2026,  ...,     0,     0,     0],
        [  101,  2017,  2467,  ...,     0,     0,     0],
        [  101,  2023,  2338,  ...,     0,     0,     0],
        ...,
        [  101,  2002,  4449,  ...,     0,     0,     0],
        [  101, 26136,  6593,  ...,     0,     0,     0],
        [  101,  1045,  2064,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [47]:
# Create DataLoader
train_dataset = TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], train_labels)
val_dataset = TensorDataset(val_inputs['input_ids'], val_inputs['attention_mask'], val_labels)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [53]:
# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [54]:
# Freeze BERT layers
for param in model.base_model.parameters():
    param.requires_grad = False

In [55]:
# Add linear classification layer on top
classifier = torch.nn.Linear(768, 2)
model.classifier = classifier

In [51]:
# Train model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [65]:
epoch_progress_bar = tqdm(range(3), desc="Epochs")

for epoch in epoch_progress_bar:
    model.train()
    # Initialize tqdm progress bar for training batches
#     train_progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{3}', leave=False)
    
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
#         train_progress_bar.set_postfix(loss=loss.item())

    
    # Evaluate model
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()
            _, predicted = torch.max(outputs.logits, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = correct / total
    print(f'Epoch {epoch+1}, Validation Loss: {val_loss/len(val_loader)}, Validation Accuracy: {accuracy}')

Epochs:  33%|████████████▎                        | 1/3 [00:22<00:44, 22.01s/it]

Epoch 1, Validation Loss: 0.5142892241477967, Validation Accuracy: 1.0


Epochs:  67%|████████████████████████▋            | 2/3 [00:48<00:24, 24.44s/it]

Epoch 2, Validation Loss: 0.5142892241477967, Validation Accuracy: 1.0


Epochs: 100%|█████████████████████████████████████| 3/3 [01:11<00:00, 23.92s/it]

Epoch 3, Validation Loss: 0.5142892241477967, Validation Accuracy: 1.0
